Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [107]:
import logging
from itertools import combinations, permutations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random
from sklearn.cluster import KMeans


from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [108]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [109]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    cluster_set = set(tsp[:-1])  # Escludi l'ultima città (uguale alla prima)
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost if cluster_set.issubset(set(range(len(CITIES)))) else float('inf')

## First Greedy Algorithm

In [110]:
def greedy(startingcity: int):
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    city = startingcity
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    tsp.append(tsp[0])
    return tsp

visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    visited[closest] = True
    city = closest
    tsp.append(int(city))
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

INFO:root:result: Found a path of 46 steps, total length 4436.03km


## Simulated annealing with 2-opt

In [111]:
# Simulated annealing parameters 
initial_temp = 0.1  # Starting temperature
cooling_rate = 0.999  # How to slow down the cooling
min_temp = 1e-5  # Minimum temperature 

def two_opt(tour):
    """Applica la mossa 2-opt per invertire una parte del percorso."""
    i, j = np.sort(np.random.randint(1, len(CITIES) - 1, size=2))
    new_tour = tour[:i] + tour[i:j+1][::-1] + tour[j+1:]
    return new_tour

def simulated_annealing(tsp, initial_temp, cooling_rate, min_temp):
    current_solution = tsp.copy()
    current_cost = tsp_cost(current_solution)
    best_solution = current_solution.copy()
    best_cost = current_cost
    temperature = initial_temp

    while temperature > min_temp:
        # Generate a new solution through 2-opt
        new_solution = two_opt(current_solution)

        # Cost of the new solution
        new_cost = tsp_cost(new_solution)

        # Cost difference
        cost_diff = new_cost - current_cost

        # Decide whether to accept the new solution
        if cost_diff < 0 or np.random.rand() < np.exp(-cost_diff / temperature):
            current_solution = new_solution
            current_cost = new_cost
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost

        # Cool down the temperature
        temperature *= cooling_rate

    return best_solution, best_cost

best_solution, best_cost = simulated_annealing(tsp, initial_temp, cooling_rate, min_temp)

logging.info(f"Simulated annealing optimized solution cost: {best_cost:.2f} km")

INFO:root:Simulated annealing optimized solution cost: 4320.03 km


In [112]:
'''
#Apply simulated annealing 15 times recursively on the solution obtained from the previous iteration
for i in range(15):
    best_solution, best_cost = simulated_annealing(best_solution, initial_temp, cooling_rate, min_temp)
    logging.info(f"Simulated annealing optimized solution cost: {best_cost:.2f} km")
'''

'\n#Apply simulated annealing 15 times recursively on the solution obtained from the previous iteration\nfor i in range(15):\n    best_solution, best_cost = simulated_annealing(best_solution, initial_temp, cooling_rate, min_temp)\n    logging.info(f"Simulated annealing optimized solution cost: {best_cost:.2f} km")\n'

## Starting point for the EA algorithm

There are different ways in which we can start the execution of the EA algorithm. In particular, given the population size, we can decide to start from a completely random population of solutions or, in the other hand, we can decide to tune the number of greedy solutions inserted inside. Moreover, to have even a better starting point, we can apply simulated annealing with 2-opt mutation (that has demonstrated to be the best one) in order to obtain even a better solution to insert inside the pool. Obviously, also in this case, there are many other things that can be improved. In particular, how many individual are greedy? How many are generated completely random? How many are greedy+simulated annealing?
For what concerns the crossover, many methods have been tried, such as order crossover, cycle crossover and so on (they are all reported in the following section). The one selected is Inver Over as it gave the best result considering only Modern GA when evaluating.
For the mutation, given the results obtained with the simulated annealing, I decided to use 2-opt again.

In [113]:
# Function to create a random solution (valid, it is a tour of the cities)
def create_random_solution():
    solution = list(range(len(CITIES)))
    np.random.shuffle(solution)
    solution.append(solution[0])  
    return solution

'''
This function receives total_size (population_size), greedy_size (number of individuals generated using the greedy algorithm), 
random_size (number of individuals randomly generated) and simulated_annealing_size (number of individuals generated 
using greedy optimized using the simulated annealing algorithm with 2-opt mutation).
'''
def create_initial_population(total_size,greedy_size,random_size,simulated_annealing_size):
    population = []
    for _ in range(greedy_size):
        population.append(greedy(np.random.randint(0,len(CITIES))))
    for _ in range(random_size):
        population.append(create_random_solution())
    for _ in range(simulated_annealing_size):
        population.append(simulated_annealing(greedy(np.random.randint(0,len(CITIES))),initial_temp,cooling_rate,min_temp)[0])
    for _ in range(total_size-len(population)):
        population.append(create_random_solution())
    return population

'''
Utility function to compute the fitness, it's simply the cost of the negative of the cost of the tour.
Since it is always a valid tour, no need to evaluate this additional constraint 
(even if there is an assert inside the tsp_cost function that has been defined by the professor)
'''
def fitness(solution):
    return -tsp_cost(solution)

# Tournament selection
def tournament_selection(population, k=30):
    selected = random.sample(population, k)
    selected.sort(key=fitness, reverse=True)
    return selected[0]

''' 
From here on, different types of crossover functions are defined (order, partially mapped, cycle, uniform and inversion).
'''

def order_crossover(parent1, parent2):
    size = len(parent1) - 1  # Exclude the last element (the return to start)
    start, end = sorted(random.sample(range(size), 2))

    child = [None] * size
    child[start:end] = parent1[start:end]  # Copy segment from parent1

    p2_index = 0
    for i in range(size):
        if child[i] is None:  # Fill in remaining positions
            while parent2[p2_index] in child:
                p2_index += 1
            child[i] = parent2[p2_index]

    child.append(child[0])  # Close the cycle
    return child

def partially_mapped_crossover(parent1, parent2):
    size = len(parent1) - 1  # Exclude the last element (the return to start)
    start, end = sorted(random.sample(range(size), 2))

    child = [None] * size
    child[start:end] = parent1[start:end]  # Copy segment from parent1

    # Create a mapping
    mapping = {}
    for i in range(start, end):
        mapping[parent1[i]] = parent2[i]

    # Fill in the remaining positions from parent2
    for i in range(size):
        if child[i] is None:
            gene = parent2[i]
            while gene in mapping:
                gene = mapping[gene]
            child[i] = gene

    child.append(child[0])  # Close the cycle
    return child

def cycle_crossover(parent1, parent2):
    size = len(parent1) - 1  # Exclude the last element (the return to start)
    child = [None] * size
    visited = [False] * size
    
    # Find cycles
    for start in range(size):
        if not visited[start]:
            current = start
            while not visited[current]:
                visited[current] = True
                child[current] = parent1[current]
                current = parent2.index(parent1[current])

    # Fill the remaining positions with genes from parent2
    for i in range(size):
        if child[i] is None:
            child[i] = parent2[i]

    child.append(child[0])  # Close the cycle
    return child

def uniform_crossover(parent1, parent2):
    size = len(parent1) - 1  # Exclude the last element (the return to start)
    mask = [random.choice([0, 1]) for _ in range(size)]

    child = [None] * size
    for i in range(size):
        if mask[i] == 0:
            child[i] = parent1[i]
        else:
            child[i] = parent2[i]

    # Fix duplicates by replacing with genes from the other parent
    for i in range(size):
        if child[i] in child:
            other_parent = parent1 if child[i] in parent2 else parent2
            for gene in other_parent:
                if gene not in child:
                    child[i] = gene
                    break

    child.append(child[0])  # Close the cycle
    return child


def inversion_crossover(parent1, parent2):
    # Determine the size of the parents (excluding the last element)
    size = len(parent1) - 1  
    # Randomly choose two crossover points
    start, end = sorted(random.sample(range(size), 2))

    # Copy the segment from parent1 and invert it
    child_segment = parent1[start:end + 1][::-1]  # Invert the selected segment
    
    # Create a child list that initially includes None
    child = [None] * size
    
    # Fill in the inverted segment into the child
    child[start:end + 1] = child_segment
    
    # Fill the remaining positions with genes from parent2, avoiding duplicates
    p2_index = 0
    for i in range(size):
        if child[i] is None:
            # Find the next gene from parent2 that isn't already in the child
            while parent2[p2_index] in child:
                p2_index += 1
            child[i] = parent2[p2_index]

    child.append(child[0])  # Close the cycle by adding the first city again
    if(random.random()<0.05): #first tests were done with 0.01
        child,_=simulated_annealing(child,initial_temp,cooling_rate,min_temp)
    return child

#2-opt
def mutate(solution, mutation_rate):
    if random.random() < mutation_rate:
        i, j = sorted(random.sample(range(1, len(solution) - 1), 2))
        solution[i:j] = reversed(solution[i:j])
    return solution

def lighter_mutation(solution, mutation_rate):
    if random.random() < mutation_rate:
        i, j = sorted(random.sample(range(1, len(solution) - 1), 2))
        solution[i], solution[j] = solution[j], solution[i]
    return solution


At this point, there are different alternative flows for the EA algorithm...
Historic GA, Modern GA  and Hyper-modern GA.
In the first case, parents are selected, mutated with probability pm and the crossover is performed with probability pc.
In the second case, parents are selected, the crossover is performed and the result of the offspring is mutated with probability pm.
In the last case, a genetic operator is selected with probability pi and then the offspring is generated.
For the selection of the parent, tournament selection has been picked as chosen method.
When creating the new population, following the Elitist strategy, the fittest can be kept inside the population. 


## First version EA-> Historical

In [114]:
# Algorithm parameters
POPULATION_SIZE = 100
GENERATIONS = 500
ELITE_SIZE = 10
MUTATION_RATE = 0.4
CROSSOVER_RATE = 0.4


'''
# EA-> Historical GA
def evolutionary_algorithm():
    #Completely random starting solution-> create_initial_population(total_size,greedy_size,random_size,simulated_annealing_size):
    population = create_initial_population(POPULATION_SIZE,int(POPULATION_SIZE*0.05),int(0.9*POPULATION_SIZE),int(0.05*POPULATION_SIZE))
    best_solution = None
    best_fitness = float('-inf')

    for generation in range(GENERATIONS):
        # Elite selection
        population.sort(key=fitness, reverse=True)
        elite = population[:ELITE_SIZE]
        
        # best individual update
        if fitness(elite[0]) > best_fitness:
            best_solution = elite[0]
            best_fitness = fitness(best_solution)

        # New population
        new_population = elite[:]  # The elite is kept
        while len(new_population) < POPULATION_SIZE:
            parent1 = mutate(tournament_selection(population),MUTATION_RATE)
            parent2 = mutate(tournament_selection(population),MUTATION_RATE)
            if(random.random() < CROSSOVER_RATE):
                child = inversion_crossover(parent1, parent2)
            else:
                #if the crossover is not performed, we randomly select one of the two parents
                child = parent1 if random.random() < 0.5 else parent2
            new_population.append(child)
        population = new_population  # Genetic
        
        if generation % 50 == 0:
            print(f"Generation {generation}: Best tour length = {-best_fitness:.2f} km")

    return best_solution, -best_fitness  

best_tour, best_cost = evolutionary_algorithm()
print(f"Best tour found: {best_tour} with total length {best_cost:.2f} km")
'''

'\n# EA-> Historical GA\ndef evolutionary_algorithm():\n    #Completely random starting solution-> create_initial_population(total_size,greedy_size,random_size,simulated_annealing_size):\n    population = create_initial_population(POPULATION_SIZE,int(POPULATION_SIZE*0.05),int(0.9*POPULATION_SIZE),int(0.05*POPULATION_SIZE))\n    best_solution = None\n    best_fitness = float(\'-inf\')\n\n    for generation in range(GENERATIONS):\n        # Elite selection\n        population.sort(key=fitness, reverse=True)\n        elite = population[:ELITE_SIZE]\n        \n        # best individual update\n        if fitness(elite[0]) > best_fitness:\n            best_solution = elite[0]\n            best_fitness = fitness(best_solution)\n\n        # New population\n        new_population = elite[:]  # The elite is kept\n        while len(new_population) < POPULATION_SIZE:\n            parent1 = mutate(tournament_selection(population),MUTATION_RATE)\n            parent2 = mutate(tournament_selection(po

## Second version EA -> Modern

In [115]:
# Algorithm parameters
POPULATION_SIZE = 150 #it was 200
GENERATIONS = 100 
ELITE_SIZE = 20  # Set to 5 to retain only the 5 best individuals-> modified to 20, try different values
MUTATION_RATE = 0.5

# EA -> Modern GA
def evolutionary_algorithm():
    # Initial population setup
    population = create_initial_population(
        POPULATION_SIZE, int(POPULATION_SIZE * 0), int(0.95 * POPULATION_SIZE), int(0.05 * POPULATION_SIZE)
    )
    best_solution = None
    best_fitness = float('-inf')

    for generation in range(GENERATIONS):
        # Sort the population by fitness in descending order
        population.sort(key=fitness, reverse=True)
        
        # Select the best individual as elite
        elite = population[:ELITE_SIZE]  # Only keep the top individual
        
        # Update the best individual if needed
        if fitness(elite[0]) > best_fitness:
            best_solution = elite[0]
            best_fitness = fitness(best_solution)

        # New population setup
        new_population = elite[:]  # Keep only the best individual
        while len(new_population) < POPULATION_SIZE:
            # Selection and crossover
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child = inversion_crossover(parent1, parent2)
            child = mutate(child, MUTATION_RATE)
            new_population.append(child)
        
        # Update population for the next generation
        population = new_population

        # Display progress every 20 generations
        if generation % 20 == 0:
            print(f"Generation {generation}: Best tour length = {-best_fitness:.2f} km")

    return best_solution, -best_fitness

# Run the evolutionary algorithm
best_tour, best_cost = evolutionary_algorithm()

print(f"Best tour found: {best_tour} with total length {best_cost:.2f} km")


Generation 0: Best tour length = 4320.03 km
Generation 20: Best tour length = 4172.76 km


KeyboardInterrupt: 

In [ ]:
best_tour, best_cost = simulated_annealing(best_tour,initial_temp,cooling_rate,min_temp)
ic(best_cost)

ic| best_cost: np.float64(35447.012832700275)


np.float64(35447.012832700275)

## Third version-> Hyper-modern

In [ ]:
# Algorithm parameters
POPULATION_SIZE = 100
GENERATIONS = 1200
ELITE_SIZE = 10
pi=0.05

# EA-> Hyper-modern GA
def evolutionary_algorithm():
    #Completely random starting solution
    population = create_initial_population(POPULATION_SIZE,int(POPULATION_SIZE*0),int(0.9*POPULATION_SIZE),int(1*POPULATION_SIZE))
    best_solution = None
    best_fitness = float('-inf')

    for generation in range(GENERATIONS):
        # Elite selection
        population.sort(key=fitness, reverse=True)
        elite = population[:ELITE_SIZE]
        
        # best individual update
        if fitness(elite[0]) > best_fitness:
            best_solution = elite[0]
            best_fitness = fitness(best_solution)

        # New population
        new_population = elite[:]  # The elite is kept
        while len(new_population) < POPULATION_SIZE:
            if(random.random() < pi):
                parent1 = tournament_selection(population)
                parent2 = tournament_selection(population)
                child = inversion_crossover(parent1, parent2)
            else:
                parent = tournament_selection(population)
                child = lighter_mutation(parent,1) #In this case the mutation has to be done
            new_population.append(child)
        
        population = new_population  # Genetic
        
        if generation % 50 == 0:
            print(f"Generation {generation}: Best tour length = {-best_fitness:.2f} km")

    return best_solution, -best_fitness  

best_tour, best_cost = evolutionary_algorithm()
print(f"Best tour found: {best_tour} with total length {best_cost:.2f} km")

Result table (Trying to report what are some of the observations when doing trials):  
1- With a complete random solution, the best result is obtained with the Modern GA, even if it seems that some more generations (more than 500) could have improved it even more. Let's try with 2000. Results for the other 2 are not good enough to be considered for other improvements. With 2000, it improves more but it reaches a plateau.  
2-Let's try to improve a little bit the starting solution. 90% are randomly generated, 5% are greedy and 5% are gredy+ simulated annealing. Also in this case, the one that performs better is the Modern GA. I'm not so able to move the results obtained in the firsty generation with the Hyper-modern, I looked for bugs in the code but it seems to be correct to me.  
3-Let's try to keep the previous configuration and the Modern approach, but I also increase the tau (highest selective pressure). I will try with: 5-10-20-30-40 and report here the one that gives the best result. 10 reaches better results with respect to 5, but it converges faster, so it looks like it won't benefit from more generations. 20 is worst than 10. 30 is the value that provides the best result so far (4174 for Italy). Let's try with higher values, 40 and 50. The best result is obtained by using 30, which is actually 30% of the population size.  
4-By keeping 30 as tau, I try to slightly increase the goodness of the starting solution. 19% greedy, 10% greedy+simulated annealing. It was better with 5%.  
5- Let's try to increase the previous percentages a lot: 30% and 30%. In this case, it reaches a good result (not the best one) but then the evolutionary algorithm is not able to improve it anymore.  
6- Let's try with a population entirely generated with greedy+simulated annealing-> the result is good also in this case, but it is only slightly moved by the EA algorithm and it's not as good as the one with 5%.  
7- Try to use 5% of greedy+simulated annealing, 95% random solutions and give the algorithm 5_000 generations. Even with more iterations, it's not able to improve it more.  
8- Try 1% simulated annealing, 99% random-> No, it's not better.  
9- Since it reaches a good value but then it's a little bit stuck on that when using 5% of simulated annealing, let's try to increase the MUTATION_RATE to favore exploitation. I will try with: 1,0.9,0.8,0.6,0.2 (one is to try to decrease it).
4248 with 0.2, 4172 with 0.6, 4195 with 0.8 (not improved by EA), 4176 with 0.5. Let's pick 0.5.  
10- By following this approach of exploitation, I have also tried the hyper-modern approach with pi=0.05 (quite always mutation) and a 'lighter' mutation wrt the 2-opt mutation, by simply swapping two cities. The starting point is the solution completely filled with results taken from greedy+simulated annealing and I kept 1200 generations-> I'm not getting a better result.  
11-Try to use 4% of the individuals of the starting population generated only by greedy, giving the algorithm more 'freedom'->